# Develop a Prototype Featureset as Style Embedding

Goal: Model the style of committers by creating a self-built style embedding including features like length, polarity, and others that were explored earlier.

#### Load Data

In [2]:
import pandas as pd

data = pd.read_pickle('data/03_Subset_Frequent_Committers.pkl')
data.head(3)

,message,committer_email,project
0,Fixed an error happening when the memory stats...,michele.simionato@gmail.com,gem_oq-engine
1,Updated setup.py [skip CI],michele.simionato@gmail.com,micheles_decorator
2,Fixed an exposure test [skip hazardlib],michele.simionato@gmail.com,gem_oq-engine


In [17]:
import numpy as np

feature_set = np.array([len(message) for message in data["message"]])

In [18]:
feature_set

array([148,  26,  39, ...,  25,  18,  58])

In [ ]:
from sklearn.preprocessing import normalize

feature_set = normalize(feature_set[:,np.newaxis], axis=0).ravel()